In [1]:
from pyspark.sql import SparkSession
import pandas

In [2]:
spark = SparkSession.builder.appName("Titanic-Dataframe").getOrCreate()
spark.sparkContext.setLogLevel('WARN')

23/12/28 17:19:11 WARN Utils: Your hostname, ikokorin-nb resolves to a loopback address: 127.0.1.1; using 192.168.0.10 instead (on interface wlp0s20f3)
23/12/28 17:19:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 17:19:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.option("header", True).csv('titanic.csv')

In [4]:
data.take(3)

[Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S'),
 Row(PassengerId='2', Survived='1', Pclass='1', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38', SibSp='1', Parch='0', Ticket='PC 17599', Fare='71.2833', Cabin='C85', Embarked='C'),
 Row(PassengerId='3', Survived='1', Pclass='3', Name='Heikkinen, Miss. Laina', Sex='female', Age='26', SibSp='0', Parch='0', Ticket='STON/O2. 3101282', Fare='7.925', Cabin=None, Embarked='S')]

In [5]:
df = pandas.read_csv('titanic.csv')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


```sql
SELECT Name, Sex, Age
FROM Titanic;
```

In [6]:
df[['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
0,"Braund, Mr. Owen Harris",male,22.0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
4,"Allen, Mr. William Henry",male,35.0


In [7]:
data.select('Name', 'Sex', 'Age').take(5)

[Row(Name='Braund, Mr. Owen Harris', Sex='male', Age='22'),
 Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', Age='26'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Allen, Mr. William Henry', Sex='male', Age='35')]

```sql
SELECT Name, Sex, Age * 12 AS AgeMonths
FROM Titanic;
```

In [8]:
df.assign(AgeMonths=df.Age * 12)[['Name', 'Sex', 'AgeMonths']].head(5)

,Name,Sex,AgeMonths
0,"Braund, Mr. Owen Harris",male,264.0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,456.0
2,"Heikkinen, Miss. Laina",female,312.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,420.0
4,"Allen, Mr. William Henry",male,420.0


In [9]:
data.withColumn('AgeMonths', data.Age * 12).select('Name', 'Sex', 'AgeMonths').take(5)

[Row(Name='Braund, Mr. Owen Harris', Sex='male', AgeMonths=264.0),
 Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', AgeMonths=456.0),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', AgeMonths=312.0),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', AgeMonths=420.0),
 Row(Name='Allen, Mr. William Henry', Sex='male', AgeMonths=420.0)]

```sql
SELECT Name, Sex, Age
FROM Titanic
WHERE Survived;
```

In [10]:
df[df.Survived == 1][['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0


In [11]:
data.filter(data.Survived == '1').select('Name', 'Sex', 'Age').take(5)

[Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', Age='26'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', Sex='female', Age='27'),
 Row(Name='Nasser, Mrs. Nicholas (Adele Achem)', Sex='female', Age='14')]

```sql
SELECT Name, Sex, Age
FROM Titanic
WHERE (Survived AND Cabin IS NOT NULL) OR Sex = 'female';
```

In [12]:
df[
    ((df.Survived == 1) & (~df.Cabin.isna())) | (df.Sex == 'female')
][['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0


In [13]:
data.filter(
    ((data.Survived == '1') & (~data.Cabin.isNull())) | (data.Sex == 'female')
).select(['Name', 'Sex', 'Age']).take(5)

[Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Heikkinen, Miss. Laina', Sex='female', Age='26'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', Sex='female', Age='27'),
 Row(Name='Nasser, Mrs. Nicholas (Adele Achem)', Sex='female', Age='14')]

```sql
SELECT Survived, avg(Age)
FROM Titanic
GROUP BY Survived
```

In [14]:
import numpy
df.groupby('Survived').aggregate({'Age': numpy.mean})

,Age
Survived,
0,30.626179
1,28.343690


In [15]:
data.groupBy('Survived').agg({'Age': 'avg'}).collect()

[Row(Survived='0', avg(Age)=30.62617924528302),
 Row(Survived='1', avg(Age)=28.343689655172415)]

```sql
SELECT Name, Sex, Age
FROM Titanic
WHERE Name LIKE '.*Mrs\.*';
```

In [16]:
df[df.Name.str.match('.*Mrs\.*')][['Name', 'Sex', 'Age']].head(5)

,Name,Sex,Age
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0
15,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0


In [17]:
data.filter(data.Name.rlike('.*Mrs\.*')).select(['Name', 'Sex', 'Age']).take(5)

[Row(Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38'),
 Row(Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35'),
 Row(Name='Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', Sex='female', Age='27'),
 Row(Name='Nasser, Mrs. Nicholas (Adele Achem)', Sex='female', Age='14'),
 Row(Name='Hewlett, Mrs. (Mary D Kingcome) ', Sex='female', Age='55')]

```sql
SELECT Survived, avg(Age)
FROM (
    SELECT Survived, Age 
    FROM Titanic 
    WHERE Name LIKE '.*Mrs\.*'
)
GROUP BY Survived;
```

In [18]:
mrs_df = df[df.Name.str.match('.*Mrs\.*')][['Survived', 'Age']]
mrs_df.groupby('Survived').aggregate({'Age': numpy.mean})

,Age
Survived,
0,35.583333
1,35.659091


In [19]:
mrs_data = data.filter(data.Name.rlike('.*Mrs\.*')).select(['Survived', 'Age'])
mrs_data.groupBy('Survived').agg({'Age': 'avg'}).collect()

[Row(Survived='0', avg(Age)=35.583333333333336),
 Row(Survived='1', avg(Age)=35.65909090909091)]

In [20]:
import pyspark.sql.functions as F

data.select(
    F.explode(
        F.split(data.Name, ' '),
    ).alias('NamePart'), 'Name', 'Age'
).take(20)

[Row(NamePart='Braund,', Name='Braund, Mr. Owen Harris', Age='22'),
 Row(NamePart='Mr.', Name='Braund, Mr. Owen Harris', Age='22'),
 Row(NamePart='Owen', Name='Braund, Mr. Owen Harris', Age='22'),
 Row(NamePart='Harris', Name='Braund, Mr. Owen Harris', Age='22'),
 Row(NamePart='Cumings,', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Age='38'),
 Row(NamePart='Mrs.', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Age='38'),
 Row(NamePart='John', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Age='38'),
 Row(NamePart='Bradley', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Age='38'),
 Row(NamePart='(Florence', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Age='38'),
 Row(NamePart='Briggs', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Age='38'),
 Row(NamePart='Thayer)', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Age='38'),
 Row(NamePart='Heikkinen,', Name='Heikkinen, Miss. Laina', Age='26')

In [47]:
from pyspark.sql.types import Row

data.rdd \
    .flatMap(lambda row: [Row(NamePart = part, Name=row.Name, age=row.Age) for part in row.Name.split(' ')]) \
    .toDF().take(20)

[Row(NamePart='Braund,', Name='Braund, Mr. Owen Harris', age='22'),
 Row(NamePart='Mr.', Name='Braund, Mr. Owen Harris', age='22'),
 Row(NamePart='Owen', Name='Braund, Mr. Owen Harris', age='22'),
 Row(NamePart='Harris', Name='Braund, Mr. Owen Harris', age='22'),
 Row(NamePart='Cumings,', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', age='38'),
 Row(NamePart='Mrs.', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', age='38'),
 Row(NamePart='John', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', age='38'),
 Row(NamePart='Bradley', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', age='38'),
 Row(NamePart='(Florence', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', age='38'),
 Row(NamePart='Briggs', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', age='38'),
 Row(NamePart='Thayer)', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', age='38'),
 Row(NamePart='Heikkinen,', Name='Heikkinen, Miss. Laina', age='26')